# Ejemplo: Taxi Autónomo

Diseñemos una simulación de un taxi autónomo. El objetivo principal es demostrar, en un entorno simplificado, cómo puede utilizar las técnicas de Aprendizaje por Refuerzo para desarrollar un enfoque eficiente y seguro para abordar este problema.

El trabajo del Taxi Autónomo es recoger al pasajero en un lugar y dejarlo en otro. Aquí hay algunas cosas de las que nos encantaría que se ocupara nuestro Taxi Autónomo:

- Dejar al pasajero en la ubicación correcta.
- Ahorre tiempo al pasajero tomando el mínimo tiempo posible para dejarlo
- Cuide las normas de seguridad y tráfico de los pasajeros.

Hay diferentes aspectos que deben considerarse aquí al modelar una solución RL para este problema: recompensas, estados y acciones.

## 1. Recompensas

Dado que el agente (el conductor imaginario) está motivado por las recompensas y va a aprender a controlar el taxi mediante experiencias de prueba en el entorno, debemos decidir las recompensas y/o sanciones y su magnitud en consecuencia. Aquí algunos puntos a considerar:

-  El agente debe recibir una alta recompensa positiva por una entrega exitosa porque este comportamiento es altamente deseado
-  El agente debe ser penalizado si intenta dejar a un pasajero en lugares incorrectos.
-  El agente debe obtener una ligera recompensa negativa por no llegar al destino después de cada intervalo de tiempo. "Ligero" negativo porque preferiríamos que nuestro agente llegara tarde en lugar de hacer movimientos incorrectos tratando de llegar al destino lo más rápido posible.

## 2. Espacio de Estados

En Aprendizaje por Refuerzo, el agente encuentra un estado y luego toma medidas de acuerdo con el estado en el que se encuentra.

El espacio de estados es el conjunto de todas las situaciones posibles que nuestro taxi podría habitar. El estado debe contener información útil que el agente necesita para tomar la acción correcta.

Digamos que tenemos un área de capacitación para nuestro Taxi donde lo estamos enseñando a transportar personas en un estacionamiento a cuatro ubicaciones diferentes $(R, G, Y, B)$:

![](../figuras/Ambiente_taxi.png)

Supongamos que el Taxi es el único vehículo en este estacionamiento. Podemos dividir el estacionamiento en una cuadrícula de $5 \times 5$, lo que nos da $25$ posibles ubicaciones de taxis. Estas $25$ ubicaciones son una parte de nuestro espacio estatal. Observe que el estado actual de la ubicación de nuestro taxi es la coordenada $(3, 1)$.

También notará que hay cuatro $(4)$ ubicaciones que podemos recoger y dejar a un pasajero: $R, G, Y, B$ o $[(0,0), (0,4), (4,0), (4,3)]$ en coordenadas de $(fila, columna)$. Nuestro pasajero ilustrado está en la ubicación $Y$ y desea ir a la ubicación $R$.

Cuando también contamos con un $(1)$ estado adicional de pasajeros dentro del taxi, podemos tomar todas las combinaciones de ubicaciones de pasajeros y destinos para llegar a un número total de estados para nuestro entorno de taxi; Hay cuatro $(4)$ destinos y cinco $(4 + 1)$ ubicaciones de pasajeros.

Entonces, nuestro entorno de taxi tiene $5 \times 5 \times 5 \times 4 = 500$. Total de estados posibles.

## 3. Espacio de Acciones

El agente se encuentra con uno de los $500$ estados y realiza una acción. La acción en nuestro caso puede ser moverse en una dirección o decidir recoger/dejar a un pasajero.

En otras palabras, tenemos seis acciones posibles:

-  sur
-  norte
-  este
-  oeste
-  recoger
-  dejar

Este es el espacio de acción: el conjunto de todas las acciones que nuestro agente puede tomar en un estado determinado.

Notarás en la ilustración de arriba que el taxi no puede realizar ciertas acciones en ciertos estados debido a las paredes. En el código del entorno, simplemente proporcionaremos una penalización de $-1$ por cada golpe en la pared y el taxi no se moverá a ningún lado. Esto solo acumulará multas y hará que el taxi considere dar la vuelta a la pared.

## Implementación con Python

Afortunadamente, [OpenAI Gym](https://gym.openai.com) tiene este entorno exacto ya construido para nosotros.

Gym ofrece diferentes entornos de juego que podemos conectar a nuestro código y probar un agente. La biblioteca expone una API para proporcionar toda la información que requeriría nuestro agente, como posibles acciones, puntaje y estado actual. Solo necesitamos enfocarnos solo en la parte del algoritmo para nuestro agente.

Utilizaremos el entorno Gym llamado `Taxi-V3`, del cual se extrajeron todos los detalles explicados anteriormente. Los objetivos, recompensas y acciones son todos iguales.

### Interfaz de Gym

Necesitamos instalar el Gym primero. Para instalar Gym ejecutar lo siguiente:

In [1]:
!pip install gym

Una vez instalado, podemos cargar el entorno del juego y ejecutar `render()` para visualizarlo:

In [3]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



La interfaz principal de Gym es `env`, que es la interfaz de entorno unificado. Los siguientes son los métodos `env` que nos serían muy útiles:

- `env.reset`: restablece el entorno y devuelve un estado inicial aleatorio.
- `env.step(acción)`: Paso del entorno por un paso de tiempo. Retorna:
   - **observación**: observaciones del medio ambiente
   - **recompensa**: si su acción fue beneficiosa o no
   - **terminado**: indica si hemos recogido y dejado a un pasajero con éxito, también llamado episodio
   - **info**: información adicional como el rendimiento y la latencia con fines de depuración
- `env.render`: renderiza un cuadro del entorno (útil para visualizar el entorno)

Nota: Estamos usando `.env` al final de `make` para evitar que el entrenamiento se detenga en 200 iteraciones, que es el valor predeterminado para la nueva versión de Gym.

Vamos a sumergirnos más en el medio ambiente.

In [5]:
env.reset() # reset environment to a new, random state
env.render()

print("Espacio de Acciones {}".format(env.action_space))
print("Espacio de Estados {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Espacio de Acciones Discrete(6)
Espacio de Estados Discrete(500)


- El cuadrado lleno representa el taxi, que es amarillo sin pasajero y verde con un pasajero.
- La símbolo ("|") representa una pared que el taxi no puede cruzar.
- R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida de pasajeros, y la letra púrpura es el destino actual.

Según lo verificado por las impresiones, tenemos un espacio de acción de tamaño 6 y un espacio de estado de tamaño 500. Como verá, nuestro algoritmo de Aprendizaje por Refuerzo no necesitará más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de manera única mediante la asignación de un número único a cada estado posible, y el Aprendizaje por Refuerzo aprende a elegir un número de acción de 0-5 donde:

- 0 = sur
- 1 = norte
- 2 = este
- 3 = oeste
- 4 = recoger
- 5 = dejar

El aprendizaje por refuerzo aprenderá un mapeo de estados a la acción óptima para realizar en ese estado mediante *exploración*, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno.

La acción óptima para cada estado es la acción que tiene la mayor recompensa acumulativa a largo plazo.

Podemos en Gym definir un estado particular y dársela al entorno para visualizar. Tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino es la ubicación 0. Usando el método de codificación de estado `Taxi-v3`, podemos hacer lo siguiente:

In [4]:
estado = env.encode(3, 1, 2, 0) # (taxi fila, taxi columna, índice pasajero, índice destino)
print("Estado:", estado)

env.s = estado
env.render()

Estado: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### La Tabla de Recompensas

Cuando se crea el entorno de Taxi, también se crea una Tabla de Recompensa inicial, llamada `P`. Podemos pensar en ella como una matriz que tiene el número de estados como filas y el número de acciones como columnas, es decir, una matriz de $estados \times acciones$.

Como cada estado está en esta matriz, podemos ver los valores de recompensa predeterminados asignados al estado de nuestro ejemplo:

In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

este diccionario tiene la estructura `{acción: [(probabilidad, siguiente_estado, recompensa, terminado)]}`.

Algunas cosas a tener en cuenta:

- El 0-5 corresponde a las acciones (sur, norte, este, oeste, recoger, dejar) que el taxi puede realizar en nuestro estado actual en la ilustración.
- En este entorno, la `probabilidad` es siempre 1.0.
- El `siguiente_estado `es el estado en el que estaríamos si tomamos la acción en este índice del dict
- Todas las acciones de movimiento tienen una recompensa de -1 y las acciones de recoger/dejar tienen una recompensa de -10 en este estado en particular. Si estamos en un estado donde el taxi tiene un pasajero y está sobre el destino correcto, veríamos una recompensa de 20 en la acción de dejar (5)
- `terminado` se usa para decirnos cuándo hemos dejado con éxito a un pasajero en la ubicación correcta. Cada entrega exitosa es el final de un **episodio**

Tenga en cuenta que si nuestro agente eligiera explorar la acción dos (2) en este estado, iría hacia el Este en una pared. El código fuente ha hecho imposible mover el taxi a través de una pared, por lo que si el taxi elige esa acción, seguirá acumulando penalizaciones de -1, lo que afecta la recompensa a largo plazo.

In [6]:
env.s = 328  # fijar el estado del ambiente de acuerdo al dibujo inicial

epocas = 0
penalidades, recompenza = 0, 0

marcos = [] # para la animación

terminado = False

while not terminado:
    accion = env.action_space.sample()
    estado, recompenza, terminado, info = env.step(accion)

    if recompenza == -10:
        penalidades += 1
    
    # Put each rendered frame into dict for animation
    marcos.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': accion,
        'reward': recompenza
        }
    )

    epocas += 1
    
    
print("Pasos de tiempo tomados: {}".format(epocas))
print("Número de penalidades: {}".format(penalidades))

Pasos de tiempo tomados: 239
Número de penalidades: 83


In [7]:
from IPython.display import clear_output
from time import sleep

def imprimir_marcos(marcos):
    for i, marco in enumerate(marcos):
        clear_output(wait=True)
        print(marco['frame'])
        print(f"Paso tiempo: {i + 1}")
        print(f"Estado: {marco['state']}")
        print(f"Acción: {marco['action']}")
        print(f"Recompenza: {marco['reward']}")
        sleep(.1)
        
imprimir_marcos(marcos)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Paso tiempo: 239
Estado: 0
Acción: 5
Recompenza: 20


No está bien. Nuestro agente toma cientos de pasos y realiza muchas dejadas incorrectas para llevar solo un pasajero al destino correcto.

Esto se debe a que no estamos *aprendiendo* de la experiencia pasada. Podemos ejecutar esto una y otra vez, y nunca se optimizará. El agente no tiene memoria de qué acción fue la mejor para cada estado, que es exactamente lo que el aprendizaje por refuerzo hará.

## Introducción al Q-learning

Vamos a utilizar un algoritmo Aprendizaje por Refuerzo simple llamado *Q-learning* que le dará algo de memoria a nuestro agente.

Esencialmente, Q-learning le permite al agente usar las recompensas del entorno para aprender, con el tiempo, la mejor acción para tomar en un estado dado.

En nuestro entorno de Taxi, tenemos la tabla de recompensas, `P`, de la cual el agente aprenderá. Hace algo al buscar recibir una recompensa por tomar una acción en el estado actual, luego actualiza un `valor Q` para recordar si esa acción fue beneficiosa.

Los valores almacenados en la tabla `Q` se denominan `valores Q` y se asignan a una combinación `(estado, acción)`.

Un valor `Q` para una combinación particular de estado-acción es representativo de la "calidad" de una acción tomada de ese estado. Los mejores `valores Q` implican mejores posibilidades de obtener mayores recompensas.

Por ejemplo, si el taxi se enfrenta a un estado que incluye a un pasajero en su ubicación actual, es muy probable que el `valor Q` para *recoger* sea más alto en comparación con otras acciones, como el *dejar* o *norte*.

Los `valores Q` se inicializan a un valor arbitrario y, a medida que el agente se expone al entorno y recibe diferentes recompensas al ejecutar diferentes acciones, los `valores Q` se actualizan utilizando la ecuación:

$$Q(estado, accion) = Q(estado, accion)+\alpha(recompensa + \gamma \hspace{0.2 cm}\underset{a}{max}\hspace{0.2 cm}Q(proximoEstado, todasAcciones) - Q(estado, accion))$$

Dónde:

- $\alpha$ (alfa) es la tasa de aprendizaje $(0 < \alpha \le 1)$ - Al igual que en entornos de aprendizaje supervisados, $\alpha$ es la medida en que nuestros `valores Q` se actualizan en cada iteración.

- $\gamma$ (gama) es el factor de descuento $(0 \le \gamma \le 1)$: determina la importancia que queremos dar a las recompensas futuras. Un valor alto para el factor de descuento (cercano a 1) captura la adjudicación efectiva a largo plazo, mientras que un factor de descuento de 0 hace que nuestro agente considere solo una recompensa inmediata, por lo que es codicioso.

### ¿Qué está diciendo esto?

Estamos asignando, o actualizando, el `valor Q` del estado y la acción actual del agente tomando primero una ponderación $(1 − \alpha)$ del `valor Q` anterior y luego agregando el valor aprendido. El valor aprendido es una combinación de la recompensa por tomar la acción actual en el estado actual, y la recompensa máxima con descuento del siguiente estado en el que estaremos una vez que tomemos la acción actual.

Básicamente, estamos aprendiendo la acción adecuada para tomar en el estado actual al observar la recompensa para el par de estado/acción actual y las recompensas máximas para el siguiente estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas juntas.

El `valor Q` de un par de estado-acción es la suma de la recompensa instantánea y la recompensa futura con descuento (del estado resultante). La forma en que almacenamos los `valores Q` para cada estado y acción es a través de una `tabla Q`.

`Tabla Q`

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Primero se inicializa a 0, y luego los valores se actualizan después del entrenamiento. Tenga en cuenta que la tabla Q tiene las mismas dimensiones que la tabla de recompensas, pero tiene un propósito completamente diferente.

### Resumiendo el proceso de Q-Learning

Desglosándolo en pasos, obtenemos

1. Inicialice la tabla Q con ceros.
2. Comience a explorar acciones: para cada estado, seleccione cualquiera de todas las acciones posibles para el estado actual (S).
3. Viaje al siguiente estado (S') como resultado de esa acción (a).
4. Para todas las acciones posibles desde el estado (S'), seleccione la que tenga el valor Q más alto.
5. Actualice los valores de la tabla Q utilizando la ecuación.
6. Establecer el siguiente estado como el estado actual.
7. Si se alcanza el estado objetivo, finalice y repita el proceso.

Explotando los valores aprendidos

Después de suficiente exploración aleatoria de acciones, los valores Q tienden a converger sirviendo a nuestro agente como una función acción-valor que puede explotar para elegir la acción más óptima de un estado dado.

Existe una compensación entre la exploración (elegir una acción aleatoria) y la explotación (elegir acciones basadas en valores Q ya aprendidos). Queremos evitar que la acción tome siempre la misma ruta y posiblemente se sobreajuste, por lo que presentaremos otro parámetro llamado $\epsilon$ "épsilon" para atender esto durante el entrenamiento.

En lugar de simplemente seleccionar la acción de valor Q mejor aprendida, a veces favoreceremos explorar más el espacio de acción. Un valor de épsilon más bajo produce episodios con más penalizaciones (en promedio), lo cual es obvio porque estamos explorando y tomando decisiones aleatorias....

## Implementando Q-learning en python

### Entrenando al agente

Primero, inicializaremos la tabla Q en una matriz de ceros de 500 × 6:

import numpy as np
tabla_q = np.zeros([env.observation_space.n, env.action_space.n])

Ahora podemos crear el algoritmo de entrenamiento que actualizará esta `tabla Q` mientras el agente explora el entorno durante miles de episodios.

En la primera parte de `while not terminado`, decidimos si elegir una acción aleatoria o explotar los valores Q ya calculados. Esto se hace simplemente usando el valor `epsilon` y comparándolo con la función `random.uniform (0, 1)`, que devuelve un número arbitrario entre 0 y 1.

Ejecutamos la acción elegida en el entorno para obtener el `proximo_estado` y la `recompensa` de realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes a `proximo_estado`, y con eso, podemos actualizar fácilmente nuestro valor Q al `nuevo_q_valor`:

In [14]:
%%time
"""Entrenando el agente"""

import random
from IPython.display import clear_output

# Hyperparámetros
alfa = 0.1
gama = 0.6
epsilon = 0.1

# Para graficar las métricas
todas_epocas = []
todas_penalidades = []

for i in range(1, 100001):
    estado = env.reset()

    epocas, penalidades, recompenza, = 0, 0, 0
    terminado = False
    
    while not terminado:
        if random.uniform(0, 1) < epsilon:
            accion = env.action_space.sample() # Explore action space
        else:
            accion = np.argmax(tabla_q[estado]) # Exploit learned values

        proximo_estado, recompenza, terminado, info = env.step(accion) 
        
        valor_anterior = tabla_q[estado, accion]
        proximo_max = np.max(tabla_q[proximo_estado])
        
        nuevo_valor = valor_anterior + alfa * (recompenza + gama * proximo_max - valor_anterior)
        tabla_q[estado, accion] = nuevo_valor

        if recompenza == -10:
            penalidades += 1

        estado = proximo_estado
        epocas += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episodio: {i}")

print("Entrenamiento finalizado.\n")

Episodio: 100000
Entrenamiento finalizado.

CPU times: user 1min 6s, sys: 13.9 s, total: 1min 20s
Wall time: 1min 5s


Ahora que la `tabla Q` se ha establecido en más de 100,000 episodios, veamos cuáles son los valores Q en el estado de nuestra ilustración:

In [16]:
tabla_q[328]

array([ -2.41637655,  -2.27325184,  -2.41593449,  -2.36233291,
       -11.23707399, -11.2861315 ])

El valor Q máximo es "norte" (-2.273), por lo que parece que Q-learning ha aprendido efectivamente la mejor acción para tomar en el estado de nuestra ilustración.

### Evaluar al agente

Vamos a evaluar el desempeño de nuestro agente. No necesitamos seguir explorando acciones, por lo que ahora la siguiente acción siempre se selecciona utilizando el mejor valor Q:

In [17]:
"""Evaluar el rendimiento del agente después del Q-learning"""

todas_epocas, todas_penalidades = 0, 0
episodios = 100

for _ in range(episodios):
    estado = env.reset()
    epocas, penalidades, recompenza = 0, 0, 0
    
    terminado = False
    
    while not terminado:
        accion = np.argmax(tabla_q[estado])
        estado, recompenza, terminado, info = env.step(accion)

        if recompenza == -10:
            penalidades += 1

        epocas += 1

    todas_penalidades += penalidades
    todas_epocas += epocas

print(f"Resultados después de {episodios} episodios:")
print(f"Promedio de pasos de tiempo por episodio: {todas_epocas / episodios}")
print(f"Promedio de penalidades por episodio: {todas_penalidades / episodios}")

Resultados después de 100 episodios:
Promedio de pasos de tiempo por episodio: 13.26
Promedio de penalidades por episodio: 0.0


Podemos ver en la evaluación que el desempeño del agente mejoró significativamente y no incurrió en penalizaciones, lo que significa que realizó las acciones correctas de recoger/dejar con 100 pasajeros diferentes.

### Referencia

- [Reinforcement Q-Learning from Scratch in Python with OpenAI Gym](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)